In [ ]:
import sys
sys.path.append('..')
import umbrella_mesh
import elastic_rods
import linkage_vis
from umbrella_mesh import UmbrellaEnergyType
from bending_validation import suppress_stdout as so
from visualization_helper import *

In [ ]:
import numpy as np
import numpy.linalg as la

### Initial Deployment

In [ ]:
name = 'sphere_cap_0.3_straight'

In [ ]:
input_path = '../../data/{}.json'.format(name)
target_mesh_path = '../../data/target_meshes/{}.obj'.format(name)

In [ ]:
# input_path = '../../data/{}.json'.format('hemisphere')
# target_mesh_path = '../../data/target_meshes/{}.obj'.format('hemisphere')

In [ ]:
from load_jsondata import read_data
input_data, io = read_data(filepath = input_path)
width = 0.5*input_data['arm_plate_edge_offset']
thickness = width * 0.5 # 1.5 mm # FIX from mm to meters everywhere
cross_section = [thickness, width]

In [ ]:
curr_um = umbrella_mesh.UmbrellaMesh(target_mesh_path, io)
curr_um.setMaterial(elastic_rods.RodMaterial('rectangle', 2000, 0.3, cross_section, stiffAxis=elastic_rods.StiffAxis.D1))

In [ ]:
curr_um.energy(UmbrellaEnergyType.Full)

#### Pin Rigid Motion



In [ ]:
use_pin = False

In [ ]:
driver = curr_um.centralJoint()
jdo = curr_um.dofOffsetForJoint(driver)
fixedVars = (list(range(jdo, jdo + 6)) if use_pin else []) + curr_um.rigidJointAngleDoFIndices()

In [ ]:
import py_newton_optimizer
OPTS = py_newton_optimizer.NewtonOptimizerOptions()
OPTS.gradTol = 1e-6
OPTS.verbose = 1
OPTS.beta = 1e-6
OPTS.niter = 300
OPTS.verboseNonPosDef = False

In [ ]:
rod_colors = []
for ri in range(curr_um.numSegments()):
    rod_colors.append(np.ones(curr_um.segment(ri).rod.numVertices()) * ri)

In [ ]:
import mesh
target_mesh = mesh.Mesh(target_mesh_path)
view = linkage_vis.LinkageViewerWithSurface(curr_um, target_mesh, width=1024, height=600)
# view = linkage_vis.LinkageViewer(curr_um, width=1024, height=600)
# view.update(scalarField = rod_colors)
set_surface_view_options(view, color = 'green', surface_color = 'gray', umbrella_transparent = False, surface_transparent = True)
view.averagedMaterialFrames = True
view.showScalarField(rod_colors)
view.show()

In [ ]:
# camParams = view.getCameraParams()

In [ ]:
# view.setCameraParams(camParams)

In [ ]:
# import time
# time.sleep(5)

In [ ]:
angles = []
def eqm_callback(prob, i):
    angles.append(curr_um.getDoFs()[curr_um.jointAngleDoFIndices()])
    if (i % 10 == 0):
        view.update()
        view.showScalarField(rod_colors)

In [ ]:
curr_um.uniformDeploymentEnergyWeight = 0.1
# curr_um.deploymentForceType = umbrella_mesh.DeploymentForceType.Constant
curr_um.targetDeploymentHeight = thickness * 5
curr_um.repulsionEnergyWeight = 0
curr_um.attractionWeight = 2000
curr_um.setHoldClosestPointsFixed(False)
curr_um.scaleInputPosWeights(0.5)

In [ ]:
curr_um.energyElastic(), curr_um.energyDeployment(), curr_um.energyRepulsion(), curr_um.energyAttraction()

In [ ]:
dof = curr_um.getDoFs()
for i in range(curr_um.numJoints()):
    if (curr_um.joint(i).jointType() == umbrella_mesh.JointType.X):
        dof[curr_um.dofOffsetForJoint(i) + 6] = 1e-3
curr_um.setDoFs(dof)

In [ ]:
angles = []
def eqm_callback(prob, i):
    angles.append(curr_um.getDoFs()[curr_um.jointAngleDoFIndices()])
    if (i % 10 == 0):
        view.update()
        view.showScalarField(rod_colors)

In [ ]:
results = umbrella_mesh.compute_equilibrium(curr_um, callback = eqm_callback, options = OPTS, fixedVars = fixedVars, elasticEnergyIncreaseFactorLimit=2.5)

In [ ]:
results.success

In [ ]:
curr_um.energyElastic(elastic_rods.EnergyType.Bend), curr_um.energyElastic(elastic_rods.EnergyType.Twist), curr_um.energyElastic(elastic_rods.EnergyType.Stretch)

### Initialize Design Optimization

In [ ]:
curr_um.energyElastic(), curr_um.energyDeployment(), curr_um.energyRepulsion(), curr_um.energyAttraction()

In [ ]:
curr_um.uniformDeploymentEnergyWeight = 50
# curr_um.deploymentForceType = umbrella_mesh.DeploymentForceType.Constant
curr_um.targetDeploymentHeight = thickness * 5
curr_um.repulsionEnergyWeight = 0
curr_um.attractionWeight = 0.1
curr_um.setHoldClosestPointsFixed(False)
curr_um.scaleInputPosWeights(0.1)

In [ ]:
import umbrella_optimization
import umbrella_optimization_finite_diff

In [ ]:
import py_newton_optimizer
opt_opts = py_newton_optimizer.NewtonOptimizerOptions()
opt_opts.gradTol = 1e-6
opt_opts.verbose = 1
opt_opts.beta = 1e-6
opt_opts.niter = 50
opt_opts.verboseNonPosDef = False

In [ ]:
results = umbrella_mesh.compute_equilibrium(curr_um, callback = eqm_callback, options = OPTS, fixedVars = fixedVars, elasticEnergyIncreaseFactorLimit=2.5)

In [ ]:
results.success

In [ ]:
optimizer = umbrella_optimization.UmbrellaOptimization(curr_um, opt_opts, 2.5, -1, False, fixedVars)

In [ ]:
optimizer.objective.terms[-1].term.activationThreshold = thickness 

In [ ]:
optimizer.beta = 5 * 1e6
optimizer.gamma = 1
optimizer.eta = 1e7

In [ ]:
optimizer.J(curr_um.getDesignParameters(), umbrella_optimization.OptEnergyType.Umbrella),  optimizer.J(curr_um.getDesignParameters(), umbrella_optimization.OptEnergyType.Target),  optimizer.J(curr_um.getDesignParameters(), umbrella_optimization.OptEnergyType.DeploymentForce)

In [ ]:
la.norm(optimizer.gradp_J(curr_um.getDesignParameters(), umbrella_optimization.OptEnergyType.Umbrella))

In [ ]:
la.norm(optimizer.gradp_J(curr_um.getDesignParameters(), umbrella_optimization.OptEnergyType.Target))

In [ ]:
la.norm(optimizer.gradp_J(curr_um.getDesignParameters(), umbrella_optimization.OptEnergyType.DeploymentForce))

In [ ]:
original_design_parameters = curr_um.getDesignParameters()

In [ ]:
rest_height_optimizer = umbrella_optimization.UmbrellaRestHeightsOptimization(optimizer)
single_rest_height_optimizer = umbrella_optimization.UmbrellaSingleRestHeightOptimization(rest_height_optimizer)

In [ ]:
import pipeline_helper, importlib
with so(): importlib.reload(pipeline_helper)

In [ ]:
from pipeline_helper import UmbrellaOptimizationCallback

In [ ]:
from umbrella_optimization import OptEnergyType


In [ ]:
algorithm = umbrella_optimization.OptAlgorithm.NEWTON_CG
cb = UmbrellaOptimizationCallback(rest_height_optimizer, curr_um, view, True, False, 1, rod_colors)

In [ ]:
rest_height_optimizer.beta, rest_height_optimizer.gamma, rest_height_optimizer.eta

In [ ]:
import time

In [ ]:
pipeline_helper.prev_time_stamp = time.time()

In [ ]:
solverStatus = umbrella_optimization.optimize(rest_height_optimizer, algorithm, 100, 1.0, 1e-2, cb, -1)

In [ ]:
solverStatus

### Visualization

In [ ]:
import importlib, pipeline_helper
importlib.reload(pipeline_helper)
from pipeline_helper import plot_objective, Visualization_Setting, get_objective_components, plot_objective_stack, get_grad_norm_components

vs = Visualization_Setting()
use_svg = False

In [ ]:
vs.figure_size = (10, 6)

In [ ]:
import matplotlib.pyplot as plt

#### Objective

In [ ]:
opt_objective_elastic, opt_objective_target, opt_objective_deployment, opt_total_objective, colors, labels = get_objective_components(cb.iterateData, vs)

In [ ]:
plot_objective(vs, opt_total_objective, 'objective.{}'.format('svg' if use_svg else 'png'), "Optimization")

In [ ]:
plot_objective_stack(vs, opt_total_objective, [opt_objective_elastic, opt_objective_target, opt_objective_deployment], colors, labels, 'objective_stack.{}'.format('svg' if use_svg else 'png'), "Optimization")

In [ ]:
objective_components_list = [opt_objective_elastic, opt_objective_target, opt_objective_deployment]
label_list = labels
color_list = colors
grad_norm = False

In [ ]:
fig, host = plt.subplots()
cmap = plt.get_cmap("Set2")
x=range(len(opt_total_objective))
y=np.array(opt_total_objective)

plt.plot(x,y)
fig.set_size_inches(vs.figure_size)
plt.ylabel('Grad Norm' if grad_norm else 'Objective Value', fontsize = vs.figure_label_size)
plt.title('Objective', fontsize = vs.figure_label_size)
fig.set_size_inches(vs.figure_size)

In [ ]:
fig, host = plt.subplots()
fig.set_size_inches(vs.figure_size)

x=range(len(opt_total_objective))
y=np.array(objective_components_list)

# Basic stacked area chart.
plt.stackplot(x,y, labels=label_list, colors = color_list, baseline='zero')
plt.legend(loc='upper right', prop={'size': 15}, fancybox=True)
plt.ylabel('Grad Norm' if grad_norm else 'Objective Value', fontsize = vs.figure_label_size)
plt.title("Optimization Objective", fontsize = vs.figure_label_size)

#### Gradient Norm

In [ ]:
opt_grad_norm_elastic, opt_grad_norm_target, opt_grad_norm_deployment, opt_total_grad_norm, colors, labels = get_grad_norm_components(cb.iterateData, vs)

In [ ]:
grad_norm_components_list = [opt_grad_norm_elastic, opt_grad_norm_target, opt_grad_norm_deployment]
label_list = labels
color_list = colors
grad_norm = True

In [ ]:
plot_objective(vs, opt_total_grad_norm, 'objective_grad_norm.{}'.format('svg' if use_svg else 'png'), "Optimization", grad_norm=grad_norm)

In [ ]:
plot_objective_stack(vs, opt_total_grad_norm, [opt_grad_norm_elastic, opt_grad_norm_target, opt_grad_norm_deployment], colors, labels, 'objective_grad_norm_stack.{}'.format('svg' if use_svg else 'png'), "Optimization Gradient Norm", grad_norm=grad_norm)

In [ ]:
fig, host = plt.subplots()
cmap = plt.get_cmap("Set2")
x=range(len(opt_total_grad_norm))
y=np.array(opt_total_grad_norm)

plt.plot(x,y)
fig.set_size_inches(vs.figure_size)
plt.ylabel('Grad Norm' if grad_norm else 'Objective Value', fontsize = vs.figure_label_size)
plt.title('Grad Norm', fontsize = vs.figure_label_size)
fig.set_size_inches(vs.figure_size)

In [ ]:
fig, host = plt.subplots()
fig.set_size_inches(vs.figure_size)

x=range(len(opt_total_grad_norm))
y=np.array(grad_norm_components_list)

# Basic stacked area chart.
plt.stackplot(x,y, labels=label_list, colors = color_list, baseline='zero')
plt.legend(loc='upper right', prop={'size': 15}, fancybox=True)
plt.ylabel('Grad Norm' if grad_norm else 'Objective Value', fontsize = vs.figure_label_size)
plt.title("Optimization Gradient Norm", fontsize = vs.figure_label_size)

In [ ]:
import numpy.linalg as la

In [ ]:
la.norm(curr_um.getDesignParameters() - rest_height_optimizer.applyTransformation(rest_height_optimizer.params()))

### Undeployment

In [ ]:
use_pin = True

In [ ]:
driver = curr_um.centralJoint()
jdo = curr_um.dofOffsetForJoint(driver)
fixedVars = (list(range(jdo, jdo + 6)) if use_pin else []) + curr_um.rigidJointAngleDoFIndices()

In [ ]:
curr_um.uniformDeploymentEnergyWeight = 0.1
# curr_um.deploymentForceType = umbrella_mesh.DeploymentForceType.Constant
curr_um.targetDeploymentHeight = thickness * 1
curr_um.repulsionEnergyWeight = 0
curr_um.attractionWeight = 0
curr_um.setHoldClosestPointsFixed(False)
curr_um.scaleInputPosWeights(0.5)

In [ ]:
curr_um.energyElastic(), curr_um.energyDeployment(), curr_um.energyRepulsion(), curr_um.energyAttraction()

In [ ]:
results = umbrella_mesh.compute_equilibrium(curr_um, callback = eqm_callback, options = OPTS, fixedVars = fixedVars, elasticEnergyIncreaseFactorLimit=2.5)

In [ ]:
results.success

In [ ]:
curr_um.energyElastic(elastic_rods.EnergyType.Bend), curr_um.energyElastic(elastic_rods.EnergyType.Twist), curr_um.energyElastic(elastic_rods.EnergyType.Stretch)

In [ ]:
curr_um.uniformDeploymentEnergyWeight = 0.

In [ ]:
results = umbrella_mesh.compute_equilibrium(curr_um, callback = eqm_callback, options = OPTS, fixedVars = fixedVars, elasticEnergyIncreaseFactorLimit=2.5)

In [ ]:
results.success

In [ ]:
curr_um.energyElastic(elastic_rods.EnergyType.Bend), curr_um.energyElastic(elastic_rods.EnergyType.Twist), curr_um.energyElastic(elastic_rods.EnergyType.Stretch)

In [ ]:
# for i in range(curr_um.numJoints()):
#     print(curr_um.joint(i).alpha)

In [ ]:
view.show()